In [1]:
import glob
import pandas as pd
from util.file_helper import get_readme_with_repo_name
from util.readme_parser import ReadmeSectionParser
from e2e_system.reproder import ClassificationReproder, _ground_truth
import os
from tqdm.auto import tqdm
import json
from datetime import datetime
import shutil


import warnings
warnings.filterwarnings("ignore")


In [2]:
keys = ["header", "parent_header", "content"]
model = "bert_base_uncased"
model_paths = glob.glob(f"model/{model}/**")
neurips_df = pd.read_csv(
    'data/paperswithcode/neurips_checklist_manuel-100.csv')
neurips_df["score"] = neurips_df.select_dtypes(bool).sum(axis=1)


In [5]:
def evaluate(reproder, m_df, group_by_parent, consecutive):
    df = m_df.copy()
    for repo in (pbar := tqdm(neurips_df.url.values, desc="Evaluating")):
        pbar.set_postfix_str(f"Evaluating {repo}")

        readme = get_readme_with_repo_name("neurips", repo)
        readme_parser = ReadmeSectionParser(repo, readme)
        sections = readme_parser.parse_sections(
            group_by_parent=group_by_parent)
        # labels = reproder.classify_sections(sections)
        # checklist, scores, _, _ = reproder.checklist(labels)
        # record = df[df["url"] == repo]
        # if consecutive:
        #     labels = reproder.merge_consecutive(pd.DataFrame(labels))

        # reprod_score = reproder.calculate_reproducibility(
        #     pd.DataFrame(labels), coeff=0.16, punishment=False)
        # df.loc[record.index,
        #        "true_reprod_score"] = df.loc[record.index, "score"] * 0.16
        # df.loc[record.index, "reprod_score"] = reprod_score
        # for k, v in checklist.items():
        #     df.loc[record.index, f"{k}_pred"] = v
        #     df.loc[record.index, f"{k}_score"] = scores[k] if v else 0
    return df


In [6]:
for model in (pbar := tqdm(model_paths)):
    pbar.set_postfix_str(f"Evaluating {model}")
    reproder = ClassificationReproder(model, keys)
    base_ev = evaluate(reproder, neurips_df,
                       group_by_parent=False, consecutive=False)
    grouped_ev = evaluate(reproder, neurips_df,
                          group_by_parent=True, consecutive=False)
    consecutive_ev = evaluate(reproder, neurips_df,
                              group_by_parent=False, consecutive=True)
    grouped_consecutive_ev = evaluate(
        reproder, neurips_df, group_by_parent=True, consecutive=True)

    # save_main_dir = f'data/paperswithcode/evaluated/classification/{model}/'
    # save_dir = os.path.join(save_main_dir, model.split("\\")[-1])
    # shutil.rmtree(save_dir, ignore_errors=True)
    # os.makedirs(save_dir, exist_ok=True)
    # info = {
    #     "model_dir": model,
    #     "keys": keys,
    #     "time": datetime.now().strftime("%Y%m%d-%H%M%S")
    # }
    # open(os.path.join(save_dir, "info.json"), 'w').write(
    #     json.dumps(info, indent=4))

    # base_ev.to_csv(f'{save_dir}/neurips_base.csv', index=False)
    # grouped_ev.to_csv(f'{save_dir}/neurips_grouped.csv', index=False)
    # consecutive_ev.to_csv(
    #     f'{save_dir}/neurips_consecutive.csv', index=False)
    # grouped_consecutive_ev.to_csv(
    #     f'{save_dir}/neurips_grouped_consecutive.csv', index=False)


  0%|          | 0/12 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 